In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/hw4

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/hw4


In [2]:
!pip install torch
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.3

In [99]:
import torch
from transformers import GPT2Tokenizer, AutoTokenizer, GPT2LMHeadModel
from datasets import load_dataset
from collections import defaultdict
import copy

## **Task 1: Byte pair encoding**

The goal is to calculate byte pair encodings for a given dataset. For a recap of the concept we refer to lecture 9, slides 22.ff and internet search.

### Task 1.1
First we want to make sure we create a suffieciently large corpus. As dataset you import the dataset "wikitext-2-raw-v1" from huggingface. Make sure to use the 500 first entries of the training dataset and subsequently filter out empty  entries. Also print the first entry of the corpus.

In [100]:
# Load WikiText-2 dataset "wikitext-2-raw-v1"
wiki_dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_dataset = wiki_dataset["train"]
corpus = []
# YOUR CODE HERE
#iterate over the first 500 entries of the trainingset and filter out empty entries
for sent in train_dataset["text"][:500]:
  if sent not in ["", " "]:
    corpus.append(sent)

print(f"First Entry == {corpus[0]}")

  0%|          | 0/3 [00:00<?, ?it/s]

First Entry ==  = Valkyria Chronicles III = 



In [101]:
corpus

[' = Valkyria Chronicles III = \n',
 ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
 " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving f

Note that the corpus should have a format like
corpus = [
    "This is the first sentence",
    "This is the second sentence",
    ..
]. For debugging during coding you can also consider working with a smaller corpus, e.g. only 5 sentences

### Task 1.2
Here, you loop through the corpus and count the word frequencies

In [102]:
# Initialize a tokenizer from the transformers library
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Initialize a defaultdict to store word frequencies
word_freqs = defaultdict(int)
# this dictionary gets entries for word_freqs[word]

alphabet = []
vocab = [""]
splits = {}
vocab_size = 200
merges = {}

# We loop through to corups to calculate word frequencies
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    # YOUR CODE HERE to count word frequencies
    #word_freqs[]
    #print(words_with_offsets)
    #print(f"NEW_WORDS == {new_words}")
    for w in new_words:
      word_freqs[w] += 1

In [54]:
#dict(sorted(word_freqs.items(), key=lambda x: x[1], reverse=True))

The following block creates the alphabet and initial vocabulary

In [58]:
aa = ["b", "A", "a", "L"]
aa.sort()

In [59]:
aa

['A', 'L', 'a', 'b']

In [103]:
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print("alphabet",alphabet)
vocab = ["<|endoftext|>"] + alphabet.copy()
splits = {word: [c for c in word] for word in word_freqs.keys()}

alphabet ['!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '¡', '¢', '£', '¤', '¥', '¦', '©', 'ª', '«', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¹', '¼', '½', '¿', 'Â', 'Ã', 'Å', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ï', 'Ċ', 'Ġ', 'Ģ', 'ģ', 'Ĥ', 'ĥ', 'Ħ', 'ħ', 'Ĩ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'Į', 'į', 'ı', 'Ĳ', 'ĳ', 'Ĵ', 'ĵ', 'Ķ', 'Ĺ', 'ĺ', 'Ļ', 'Ľ', 'Ł', 'ł', 'Ń']


In [104]:
splits

{'Ġ=': ['Ġ', '='],
 'ĠValkyria': ['Ġ', 'V', 'a', 'l', 'k', 'y', 'r', 'i', 'a'],
 'ĠChronicles': ['Ġ', 'C', 'h', 'r', 'o', 'n', 'i', 'c', 'l', 'e', 's'],
 'ĠIII': ['Ġ', 'I', 'I', 'I'],
 'ĠĊ': ['Ġ', 'Ċ'],
 'ĠSenjÅį': ['Ġ', 'S', 'e', 'n', 'j', 'Å', 'į'],
 'Ġno': ['Ġ', 'n', 'o'],
 'Ġ3': ['Ġ', '3'],
 'Ġ:': ['Ġ', ':'],
 'ĠUnrecorded': ['Ġ', 'U', 'n', 'r', 'e', 'c', 'o', 'r', 'd', 'e', 'd'],
 'Ġ(': ['Ġ', '('],
 'ĠJapanese': ['Ġ', 'J', 'a', 'p', 'a', 'n', 'e', 's', 'e'],
 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': ['Ġ',
  'æ',
  'Ī',
  '¦',
  'å',
  'ł',
  '´',
  'ã',
  'ģ',
  '®',
  'ã',
  'ĥ',
  '´',
  'ã',
  'Ĥ',
  '¡',
  'ã',
  'ĥ',
  '«',
  'ã',
  'Ĥ',
  'Ń',
  'ã',
  'ĥ',
  '¥',
  'ã',
  'ĥ',
  'ª',
  'ã',
  'Ĥ',
  '¢'],
 '3': ['3'],
 'Ġ,': ['Ġ', ','],
 'Ġlit': ['Ġ', 'l', 'i', 't'],
 'Ġ.': ['Ġ', '.'],
 'Ġof': ['Ġ', 'o', 'f'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠBattlefield': ['Ġ', 'B', 'a', 't', 't', 'l', 'e', 'f', 'i', 'e', 'l', 'd'],
 'Ġ)': ['Ġ', ')'],
 'Ġcommonly': ['Ġ', 'c', 'o', 'm', 'm', 'o'

### Task 1.3
Define a function to compute the frequency of each pair of characters

In [62]:
list(zip(splits["Ġblack"], splits["Ġblack"][:1]))

[('Ġ', 'Ġ')]

In [105]:
# Define a function to compute the frequency of each pair of characters
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    # YOUR CODE HERE to compute the frequency of each pair of characters
    for k, v in splits.items():
      #v is a list of characters
      for i in range(len(v) - 1):
        pair = (v[i], v[i+1])
        pair_freqs[pair] += 1
    return pair_freqs

# Initialize pair_freqs by calling the compute_pair_freqs function
pair_freqs = compute_pair_freqs(splits)

In [125]:
sorted(pair_freqs.items(), key=lambda x: x[1], reverse=True)[:50]

[(('i', 'n'), 582),
 (('e', 'r'), 525),
 (('e', 'd'), 522),
 (('o', 'n'), 467),
 (('t', 'e'), 422),
 (('r', 'e'), 420),
 (('t', 'i'), 401),
 (('e', 's'), 377),
 (('e', 'n'), 365),
 (('Ġ', 's'), 362),
 (('n', 'g'), 352),
 (('a', 't'), 329),
 (('a', 'n'), 322),
 (('a', 'l'), 311),
 (('o', 'r'), 305),
 (('s', 't'), 301),
 (('a', 'r'), 296),
 (('Ġ', 'c'), 283),
 (('n', 't'), 275),
 (('Ġ', 'p'), 269),
 (('d', 'e'), 252),
 (('l', 'e'), 244),
 (('s', 'e'), 241),
 (('r', 'i'), 235),
 (('i', 'o'), 230),
 (('r', 'a'), 230),
 (('Ġ', 'a'), 224),
 (('c', 'o'), 208),
 (('i', 't'), 208),
 (('i', 's'), 205),
 (('Ġ', 'r'), 199),
 (('v', 'e'), 195),
 (('i', 'c'), 191),
 (('n', 'e'), 188),
 (('Ġ', 'd'), 186),
 (('r', 'o'), 183),
 (('l', 'i'), 179),
 (('e', 'l'), 179),
 (('m', 'e'), 177),
 (('e', 'a'), 173),
 (('l', 'a'), 169),
 (('n', 's'), 164),
 (('s', 'i'), 163),
 (('Ġ', 'm'), 162),
 (('n', 'd'), 160),
 (('Ġ', 't'), 159),
 (('t', 'a'), 159),
 (('c', 'e'), 158),
 (('l', 'l'), 157),
 (('Ġ', 'f'), 155)]

Here we find the pair of characters that appears most frequently together

In [108]:
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print("\nMost frequent pair of characters : ", best_pair, max_freq)

('Ġ', '='): 1
('Ġ', 'V'): 29
('V', 'a'): 10
('a', 'l'): 311
('l', 'k'): 9
('k', 'y'): 5

Most frequent pair of characters :  ('i', 'n') 582


### Task 1.4
Define a function to merge the most frequent pair of characters

In [123]:
def help_fct(a, b, split):
  merged_list = []
  i = 0
  while i < len(split):
    if split[i] == a and i + 1 < len(split) and split[i + 1] == b:
      merged_list.append(a + b)
      i += 2
    else:
      merged_list.append(split[i])
      i += 1

  return merged_list

In [124]:
#tests
lst = ['c', 'a', 'b', 'd', 'e', 'f', 'a', 'b', 'g', 'h']
lst2 = "e n t e r t a i n i n g".split(" ")
help_fct(a="i", b="n", split=lst2)

['e', 'n', 't', 'e', 'r', 't', 'a', 'in', 'in', 'g']

In [120]:

# Define a function to merge the most frequent pair of characters
def merge_pair(a, b, splits):
    splits_copy = copy.deepcopy(splits)
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        # YOUR CODE HERE to merge the most frequent pair of characters in each word
        splits_copy[word] = help_fct(a=a, b=b, split=split)
    return splits_copy

In [121]:
splits["Ġserving"]

['Ġ', 's', 'e', 'r', 'v', 'i', 'n', 'g']

In [122]:
merge_pair(a="i", b="n", splits=splits)

{'Ġ=': ['Ġ', '='],
 'ĠValkyria': ['Ġ', 'V', 'a', 'l', 'k', 'y', 'r', 'i', 'a'],
 'ĠChronicles': ['Ġ', 'C', 'h', 'r', 'o', 'n', 'i', 'c', 'l', 'e', 's'],
 'ĠIII': ['Ġ', 'I', 'I', 'I'],
 'ĠĊ': ['Ġ', 'Ċ'],
 'ĠSenjÅį': ['Ġ', 'S', 'e', 'n', 'j', 'Å', 'į'],
 'Ġno': ['Ġ', 'n', 'o'],
 'Ġ3': ['Ġ', '3'],
 'Ġ:': ['Ġ', ':'],
 'ĠUnrecorded': ['Ġ', 'U', 'n', 'r', 'e', 'c', 'o', 'r', 'd', 'e', 'd'],
 'Ġ(': ['Ġ', '('],
 'ĠJapanese': ['Ġ', 'J', 'a', 'p', 'a', 'n', 'e', 's', 'e'],
 'ĠæĪ¦åł´ãģ®ãĥ´ãĤ¡ãĥ«ãĤŃãĥ¥ãĥªãĤ¢': ['Ġ',
  'æ',
  'Ī',
  '¦',
  'å',
  'ł',
  '´',
  'ã',
  'ģ',
  '®',
  'ã',
  'ĥ',
  '´',
  'ã',
  'Ĥ',
  '¡',
  'ã',
  'ĥ',
  '«',
  'ã',
  'Ĥ',
  'Ń',
  'ã',
  'ĥ',
  '¥',
  'ã',
  'ĥ',
  'ª',
  'ã',
  'Ĥ',
  '¢'],
 '3': ['3'],
 'Ġ,': ['Ġ', ','],
 'Ġlit': ['Ġ', 'l', 'i', 't'],
 'Ġ.': ['Ġ', '.'],
 'Ġof': ['Ġ', 'o', 'f'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'ĠBattlefield': ['Ġ', 'B', 'a', 't', 't', 'l', 'e', 'f', 'i', 'e', 'l', 'd'],
 'Ġ)': ['Ġ', ')'],
 'Ġcommonly': ['Ġ', 'c', 'o', 'm', 'm', 'o'

### Task 1.5

Iterate this process until you reach a predefined vocabulary size

In [ ]:
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size



We provide the tokenize function to you

In [126]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])
tokenize("Darmstadt holds the official title City of Science (German: Wissenschaftsstadt) as it is a major centre of scientific institutions, universities, and high-technology companies.")

['D',
 'a',
 'r',
 'm',
 's',
 't',
 'a',
 'd',
 't',
 'Ġ',
 'h',
 'o',
 'l',
 'd',
 's',
 'Ġ',
 't',
 'h',
 'e',
 'Ġ',
 'o',
 'f',
 'f',
 'i',
 'c',
 'i',
 'a',
 'l',
 'Ġ',
 't',
 'i',
 't',
 'l',
 'e',
 'Ġ',
 'C',
 'i',
 't',
 'y',
 'Ġ',
 'o',
 'f',
 'Ġ',
 'S',
 'c',
 'i',
 'e',
 'n',
 'c',
 'e',
 'Ġ',
 '(',
 'G',
 'e',
 'r',
 'm',
 'a',
 'n',
 ':',
 'Ġ',
 'W',
 'i',
 's',
 's',
 'e',
 'n',
 's',
 'c',
 'h',
 'a',
 'f',
 't',
 's',
 's',
 't',
 'a',
 'd',
 't',
 ')',
 'Ġ',
 'a',
 's',
 'Ġ',
 'i',
 't',
 'Ġ',
 'i',
 's',
 'Ġ',
 'a',
 'Ġ',
 'm',
 'a',
 'j',
 'o',
 'r',
 'Ġ',
 'c',
 'e',
 'n',
 't',
 'r',
 'e',
 'Ġ',
 'o',
 'f',
 'Ġ',
 's',
 'c',
 'i',
 'e',
 'n',
 't',
 'i',
 'f',
 'i',
 'c',
 'Ġ',
 'i',
 'n',
 's',
 't',
 'i',
 't',
 'u',
 't',
 'i',
 'o',
 'n',
 's',
 ',',
 'Ġ',
 'u',
 'n',
 'i',
 'v',
 'e',
 'r',
 's',
 'i',
 't',
 'i',
 'e',
 's',
 ',',
 'Ġ',
 'a',
 'n',
 'd',
 'Ġ',
 'h',
 'i',
 'g',
 'h',
 '-',
 't',
 'e',
 'c',
 'h',
 'n',
 'o',
 'l',
 'o',
 'g',
 'y',
 'Ġ',
 'c'

### Task 1.6
Now, repeat the byte pair encoding for vocab sizes of 1000 and 5000. What do you notice?

In [ ]:
# Use this chunk for the answer

In [ ]:
# Use this chunk for the answer

# **Task 2: Getting to Know Foundation Models**

In this task, we want to load two different versions of GPT-2 and compare their performance.

### Task 2.1

After we imported transformers from Huggingface, your first task is to initialize the GPT2LMHead Model. This model should be a gpt2 model from huggingface under the variable model. Also define the GPT2 tokenizer.

While we have seen the transformers in the lectures, you should print a single GPT2Block to the console for the 10th layer. Then, count the parameters of the model.

In [ ]:
# Print a GPT2Block in the 10th layer

def count_parameters(model):
    #YOUR CODE HERE
    pass

# GPT2_small parameter trainable parameters:
print(f"The model has {count_parameters(model_small):,} trainable parameters")

### Task 2.2
Because we want to compare two versions of GPT, you now also load the gpt2-large from huggingface as model. As evaluation dataset you import the dataset "wikitext-2-raw-v1" from huggingface. (like in Task1) Make sure to use the 200 first entries of the test dataset and print an entry. Also print the parameters of the gpt2-large model

In [ ]:
# Load the gpt2-large model from huggingface

# Load the dataset

# Print any single entry

# Print Trainable parameters of huggingface gpt2-large

### Task 2.3
As a next step, we define an evaluation metric to calculate predictions. Remember the perplexity funciton from the lecture. The function should take a model, tokenizer and a text and return the perplexity score for the given text.

Hint: you should encode the text, generate outputs for inputs and corresponding labels and then calculate perplexity.

In [ ]:
def calculate_perplexity(model, tokenizer, text):
    # YOUR CODE HERE

In [ ]:
perplexities_small = []
perplexities_large = []
# Loop over the dataset, append scores to the lists

# Calculate and print average perplexities
print("Average perplexity (GPT2 ): ", np.nanmean(perplexities_small))
print("Average perplexity (GPT2 large): ", np.nanmean(perplexities_large))

What do you notice? Please explain.

### Task 2.4 Masking attention
What is an attention mask? When and why is it usually used?

Your answer here:

Your task is now to run a prompt in regular inference, but with an attention mask. You can reuse the gpt2_small model

In [ ]:
text_prompt = "The sun was setting behind"

# Step 1: Implement attention masking
def apply_attention_mask(input_ids, mask_start_idx, mask_end_idx):
    # -----------------------
    #YOUR CODE HERE
    # -----------------------

    #return attention_mask

# Step 2: Apply attention masks
mask_start = 3  # Start index of the region to mask
mask_end = 5  # End index of the region to mask
input_ids = tokenizer_small.encode(text_prompt, return_tensors='pt')
attention_mask = apply_attention_mask(input_ids, mask_start, mask_end)

# Step 3: Generate output using modified attention mechanism
with torch.no_grad():
    outputs = model_small.generate(input_ids, attention_mask=attention_mask)
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("Text Prompt:", text_prompt)
print("Generated Text:", generated_text)



In [ ]:
# Step 5: Run the same prompt without attention mask

# YOUR CODE HERE

Step 6: Compare the two outputs and describe it in one to two sentences

Your answer here: